In [1]:
#Linear Regression

import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
inputs = np.load("data/classification/inputs.npy")
labels = np.load("data/classification/labels.npy")
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, random_state=42)

## Logistic Regression

In [3]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


logisticReg = LogisticRegression(random_state=0).fit(X_train, np.ravel(y_train))
y_pred = logisticReg.predict(X_test)
log_reg_acc = accuracy_score(y_pred, y_test)

## SVM

In [4]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svm = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5)).fit(X_train, np.ravel(y_train))
y_pred = svm.predict(X_test)
svm_acc = accuracy_score(y_pred, y_test)

## SVC

In [5]:
from sklearn.svm import SVC

svc = make_pipeline(StandardScaler(), SVC(gamma='auto')).fit(X_train, np.ravel(y_train))
y_pred = svc.predict(X_test)
svc_acc = accuracy_score(y_pred, y_test)

## NuSVC

In [6]:
from sklearn.svm import NuSVC

nusvc = make_pipeline(StandardScaler(), NuSVC(nu=0.3)).fit(X_train, np.ravel(y_train))
y_pred = nusvc.predict(X_test)
nusvc_acc = accuracy_score(y_pred, y_test)

In [7]:
from sklearn.model_selection import cross_val_score
import warnings; warnings.simplefilter('ignore')

log_reg_score = cross_val_score(logisticReg, inputs, np.ravel(labels), cv=5)
svm_score = cross_val_score(svm, inputs, labels.ravel(), cv=5)
svc_score = cross_val_score(svc, inputs, labels.ravel(), cv=5)
nusvc_score = cross_val_score(nusvc, inputs, labels.ravel(), cv=5)

print("Cross Validation Score:")
print(f"Logistic Regression: {log_reg_score}")
print(f"SVM: {svm_score}")
print(f"SVC: {svc_score}")
print(f"NuSVC: {nusvc_score}")

Cross Validation Score:
Logistic Regression: [0.87  0.895 0.86  0.915 0.905]
SVM: [0.87  0.895 0.865 0.91  0.905]
SVC: [0.89  0.875 0.845 0.91  0.905]
NuSVC: [0.89  0.87  0.855 0.895 0.895]


In [8]:
print("Accuracy Score:")
print(f"Logistic Regression: {log_reg_acc}")
print(f"SVM: {svm_acc}")
print(f"SVC: {svc_acc}")
print(f"NuSVC: {nusvc_acc}")

Accuracy Score:
Logistic Regression: 0.91
SVM: 0.905
SVC: 0.905
NuSVC: 0.89


## Optimization Method

In [9]:
from skopt.space import Categorical, Real, Integer
import skopt

search_space = list()
##search_space.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
# search_space.append(Integer(1, 5, name='degree'))
# search_space.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))

In [10]:

from sklearn.model_selection import RepeatedStratifiedKFold
from skopt.utils import use_named_args

# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
	# configure the model with specific hyperparameters
	model = SVC()
	model.set_params(**params)
	# define test harness
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# calculate 5-fold cross validation
	result = cross_val_score(model, inputs, np.ravel(labels), cv=cv, n_jobs=-1, scoring='accuracy')
	# calculate the mean of the scores
	estimate = np.mean(result)
	# convert from a maximizing score to a minimizing score
	return 1.0 - estimate

In [11]:
from skopt import gp_minimize
import warnings; warnings.simplefilter('ignore')

# perform optimization
result = gp_minimize(evaluate_model, search_space)
# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: %s' % (result.x))

Best Accuracy: 0.888
Best Parameters: ['linear']
